In [1]:
import json, codecs
import numpy as np

_vector_encoder_train = json.loads(codecs.open('vector_encoder_train.json', 'r', encoding='utf-8').read())
vector_encoder_train = np.array(_vector_encoder_train)

_vector_encoder_test = json.loads(codecs.open('vector_encoder_test.json', 'r', encoding='utf-8').read())
vector_encoder_test = np.array(_vector_encoder_test)

_X_train = json.loads(codecs.open('X_train.json', 'r', encoding='utf-8').read())
X_train = [np.array(_X_train)]

_X_test = json.loads(codecs.open('X_test.json', 'r', encoding='utf-8').read())
X_test = [np.array(_X_test)]

y_train = json.loads(codecs.open('y_train.json', 'r', encoding='utf-8').read())

y_test = json.loads(codecs.open('y_test.json', 'r', encoding='utf-8').read())

#### Shaping the data

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten
import numpy as np
tf.config.list_physical_devices('GPU')

Y_train = np.array(y_train, dtype=np.int32)
y_train_of_zeros = np.zeros((Y_train.shape[0], 3),dtype=np.float32)
y_train_of_zeros[np.arange(Y_train.shape[0]), Y_train] = 1.0
train_labels = y_train_of_zeros.reshape(y_train_of_zeros.shape[0],1,y_train_of_zeros.shape[1])


Y_test = np.array(y_test, dtype=np.int32)
y_test_of_zeros = np.zeros((Y_test.shape[0], 3),dtype=np.float32)
y_test_of_zeros[np.arange(Y_test.shape[0]), Y_test] = 1.0
valid_test_labels = y_test_of_zeros.reshape(y_test_of_zeros.shape[0],1,y_test_of_zeros.shape[1])



train_set = vector_encoder_train.reshape(vector_encoder_train.shape[0],
                                                             vector_encoder_train.shape[1],1)
valid_test_set = np.asarray(vector_encoder_test).reshape(vector_encoder_test.shape[0],
                                                            vector_encoder_test.shape[1],1)

validation_labels = valid_test_labels[:100]
test_labels = valid_test_labels[100:]

validation_set = valid_test_set[:100]
test_set = valid_test_set[100:]

#### Modeling

In [ ]:
model = Sequential()
model.add(Conv1D(256,kernel_size=train_set.shape[1],activation='relu',
                            input_shape = (15,1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))

model.add(Conv1D(256,kernel_size=1,activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))


model.add(Conv1D(256,kernel_size=1,activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(Dense(3,activation='relu'))

model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(train_set, train_labels, batch_size=100, epochs=100)
print('done')


In [ ]:
results = model.evaluate(validation_set, validation_labels, batch_size=100)
print("test loss, test acc:", results)
predicted_labels = (model.predict(test_set) > 0.5).astype("float32")
print("test set predictions:", predicted_labels)
model.save('documents-CNN_2.model')